In [1]:
from __future__ import print_function
from matplotlib import pyplot as plt
# get_ipython().magic(u'matplotlib auto')
import tensorflow as tf
# import tensorflow.compat.v1 as tf
# tf.disable_v2_behavior()
# import torch
from tensorflow import keras
import statistics

In [2]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Flatten, Reshape
from tensorflow.keras.layers import Conv2D, MaxPooling2D,Activation
from tensorflow import reshape
from keras.utils import np_utils
from tensorflow.keras.layers import Conv1D,Conv2D, MaxPooling1D,AveragePooling1D
from tensorflow.keras.layers import SeparableConv1D, Bidirectional
from tensorflow.keras.layers import LocallyConnected2D
from tensorflow.keras.layers import ZeroPadding2D,ZeroPadding1D, MaxPooling2D, Bidirectional
from tensorflow.keras.regularizers import l2,l1
from tensorflow.keras.layers import BatchNormalization
from tensorflow.keras.callbacks import CSVLogger,LearningRateScheduler
from tensorflow.keras.callbacks import EarlyStopping,ModelCheckpoint

In [3]:
from scipy.io import loadmat
import numpy as np
import pandas as pd
import seaborn as sns
from numpy import mean
from numpy import std
# import coremltools
# from torch import nn, optim
# import torch.nn.functional as F
import scipy.io
from scipy import stats
#from IPython.display import display, HTML
from tensorflow.keras import optimizers
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import GRU
from tensorflow.keras.layers import TimeDistributed
from sklearn import metrics
from sklearn.metrics import classification_report
from sklearn import preprocessing

In [4]:
# The number of steps within one time segment
TIME_PERIODS = 150
# The steps to take from one segment to the next; if this value is equal to
# TIME_PERIODS, then there is no overlap between the segments
STEP_DISTANCE = 150
N_FEATURES = 8

In [5]:
column_names = ['C'+str(j) for j in range(1, N_FEATURES+1)]
lst = ['Class_label']
column_names = column_names+lst
print(column_names)

['C1', 'C2', 'C3', 'C4', 'C5', 'C6', 'C7', 'C8', 'Class_label']


In [6]:
def read_data_Train(file_path_Train):
    df_Train = pd.read_csv(file_path_Train,header=None,names=column_names)
    # Last column has a ";" character which must be removed ...
    df_Train['Class_label'].replace(regex=True,inplace=True,to_replace=r';',value=r'')
    # ... and then this column must be transformed to float explicitly
    df_Train['Class_label'] = df_Train['Class_label'].apply(convert_to_float)
    # This is very important otherwise the model will not fit and loss
    # will show up as NAN
    df_Train.dropna(axis=0, how='any', inplace=True)
    return df_Train
def convert_to_float(x):
    try:
        return np.float(x)
    except:
        return np.nan

In [7]:
def show_basic_dataframe_info(dataframe):
    # Shape and how many rows and columns
    print('Number of columns in the dataframe: %i' % (dataframe.shape[1]))
    print('Number of rows in the dataframe: %i\n' % (dataframe.shape[0]))

In [8]:
def read_data_Test(file_path_Test):
    df_Test = pd.read_csv(file_path_Test,header=None,names=column_names)
    # Last column has a ";" character which must be removed ...
    df_Test['Class_label'].replace(regex=True,inplace=True,to_replace=r';',value=r'')
    # ... and then this column must be transformed to float explicitly
    df_Test['Class_label'] = df_Test['Class_label'].apply(convert_to_float)
    # This is very important otherwise the model will not fit and loss
    # will show up as NAN
    df_Test.dropna(axis=0, how='any', inplace=True)
    return df_Test

In [9]:
 pd.options.display.float_format = "{:,.5f}".format

In [10]:
# 15 function to segment data into trial lengths (trial length =513 samples in this dataset)
def create_segments_and_labels(df, time_steps,step,n_features, label_name):
    segments = []
    labels = []
    for i in range(0, len(df) - time_steps, step):
      for j in range(1, n_features+1):
        L = ('C'+str(j)) 
        segments.append(df[str(L)].values[i: i + time_steps])
      label = stats.mode(df[label_name][i: i + time_steps])[0][0]
      labels.append(label)
    reshaped_segments = np.asarray(segments, dtype= np.float32).reshape(-1, time_steps, n_features)
    labels = np.asarray(labels)
    return reshaped_segments, labels

In [11]:
verbose, epochs, batch_size = 0, 160, 512

In [12]:
# def step_decay(epoch):
#    initial_lrate = 0.1
#    drop = 0.1
#    epochs_drop = 16.0
#    lrate = initial_lrate * tf.math.pow(drop,  
#            tf.math.floor((1+epoch)/epochs_drop))
#    return lrate

In [13]:
def step_decay(epoch):
   initial_lrate = 1e-3
   drop = 0.1
   epochs_drop = 40.0
   lrate = initial_lrate * tf.math.pow(drop,  
           tf.math.floor((1+epoch)/epochs_drop))
   return lrate

In [14]:
lrate = tf.keras.callbacks.LearningRateScheduler(step_decay)

In [19]:
# lrate = tf.keras.callbacks.LearningRateScheduler(step_decay)
test_acc=[]

In [20]:
# import os
path='/media/naveen/nav/mat_codes/BioPatRec/olsson_prep_WOR_BP_D2/sub_wise_process_TT/'

In [21]:
for i in range(1,18):
    file_path_train=path+'S'+str(i)+'_'+'tr'+'.'+'csv'
    print(file_path_train)
    df_Train=read_data_Train(file_path_train)
    show_basic_dataframe_info(df_Train)
    df_Train.head(5)
    file_path_test=path+'S'+str(i)+'_'+'tt'+'.'+'csv'
    print(file_path_test)
    df_Test=read_data_Test(file_path_test)
    show_basic_dataframe_info(df_Test)
    df_Test.head(5)
    
    scaler = preprocessing.StandardScaler().fit(df_Train.iloc[:,0:N_FEATURES])
    df_Train.iloc[:,0:N_FEATURES]=scaler.transform(df_Train.iloc[:,0:N_FEATURES])
    df_Test.iloc[:,0:N_FEATURES]=scaler.transform(df_Test.iloc[:,0:N_FEATURES])
       
    LABEL = 'ActivityEncoded'
    # Transform the labels from String to Integer via LabelEncoder
    le = preprocessing.LabelEncoder()
    # Add a new column to the existing DataFrame with the encoded values
    df_Train[LABEL] = le.fit_transform(df_Train['Class_label'].values.ravel())
    # df_Valid[LABEL] = le.fit_transform(df_Valid['Class_label'].values.ravel())
    df_Test[LABEL] = le.fit_transform(df_Test['Class_label'].values.ravel())
    x_train, y_train = create_segments_and_labels(df_Train,TIME_PERIODS,STEP_DISTANCE,N_FEATURES,LABEL)
    print('x_train shape: ', x_train.shape)
    # print(x_train)
    print(x_train.shape[0], 'training samples')
    print('y_train shape: ', y_train.shape)
    # Set input & output dimensions
    num_time_periods, num_sensors = x_train.shape[1], x_train.shape[2]
    print('num_time_periods',num_time_periods)
    print('num_sensors',num_sensors)
    num_classes = le.classes_.size
    print('class_list',list(le.classes_))
    # input_shape = (num_time_periods,num_sensors)
    # print(input_shape)
    input_shape = (num_time_periods,num_sensors)
    #x_train = x_train.reshape(x_train.shape[0], input_shape)
    print('x_train shape:', x_train[0].shape)
    print('input_shape:', input_shape)
    x_train = x_train.astype('float32')
    # x_train = [torch.tensor(arr, dtype=torch.float32) for arr in x_train]
    # y_train = y_train.astype('float32')
    print('y_train shape',y_train.shape)
    y_train_hot = np_utils.to_categorical(y_train, num_classes)
#     print(y_train_hot)
    # y_train_hot= [torch.tensor(arr, dtype=torch.uint8) for arr in y_train_hot]
    print('New y_train shape: ', y_train_hot.shape)
    x_test, y_test = create_segments_and_labels(df_Test,TIME_PERIODS,STEP_DISTANCE,N_FEATURES,LABEL)
    print('x_test shape: ', x_test.shape)
    # print(x_train)
    print(x_test.shape[0], 'testing samples')
    print('y_test shape: ', y_test.shape)
    # Set input_shape / reshape for Keras
    #x_test = x_test.reshape(x_test.shape[0], input_shape)
    x_test = x_test.astype('float32')
    y_test = y_test.astype('float32')
    y_test_hot = np_utils.to_categorical(y_test, num_classes)
    # n_steps, n_length = 20, 25
    # n_steps, n_length= 10, 50
    # n_steps, n_length= 16, 32
    n_steps, n_length = 150, 8
    n_depth=1
    x_train = x_train.reshape(x_train.shape[0], n_steps, n_length,n_depth)
    print('x_train shape: ', x_train.shape)
    # x_valid = x_valid.reshape(x_valid.shape[0], n_steps, n_length, n_depth)
    # print('x_valid shape: ', x_valid.shape)
    x_test = x_test.reshape(x_test.shape[0], n_steps, n_length,n_depth)
    print('x_test shape: ', x_test.shape)
    n_outputs = y_train_hot.shape[1]
    print('n_outputs',n_outputs)
    model = Sequential()
    model.add(Conv2D(filters=64, kernel_size=(9,1), padding='same', kernel_initializer="he_normal",strides=(3,1),kernel_regularizer=l1(1e-04), \
                           input_shape=(n_steps,n_length,n_depth)))
    model.add(BatchNormalization(epsilon=1e-06, momentum=0.95, weights=None))
    model.add(MaxPooling2D(pool_size=(8,1),strides=(2,1)))
    model.add(Activation('tanh'))
    model.add(Conv2D(filters=64, kernel_size=(5,1),padding="same",kernel_initializer="he_normal",strides=(2,1),kernel_regularizer=l1(1e-04)))
    model.add(BatchNormalization(epsilon=1e-06, momentum=0.95, weights=None))
    model.add(Activation('tanh'))
    model.add(Dropout(0.2093))
    model.add(Activation('relu'))
    model.add(Flatten())
    model.add(Dense(512, activation='tanh',kernel_initializer="he_normal"))
    model.add(BatchNormalization(epsilon=1e-06, momentum=0.95, weights=None))
    model.add(Activation('tanh'))
    model.add(Dense(n_outputs, activation='softmax',kernel_initializer="he_normal"))
#     sgd=optimizers.SGD(learning_rate=2.17e-3, momentum=0.9, nesterov=False, name='SGD')
#     model = Sequential()
#     model.add(TimeDistributed(Conv1D(filters=64, kernel_size=9, padding='same', kernel_initializer="he_normal",strides=2,kernel_regularizer=l1(1e-04)), \
#                            input_shape=(n_steps,n_length,n_depth)))
#     # model.add(TimeDistributed(BatchNormalization(epsilon=1e-06, momentum=0.95, weights=None)))
#     model.add(TimeDistributed(MaxPooling1D(pool_size=4,strides=2)))
#     model.add(TimeDistributed(Activation('tanh')))
#     model.add(TimeDistributed(Conv1D(filters=64, kernel_size=5,padding="same",kernel_initializer="he_normal",strides=2,kernel_regularizer=l1(1e-04))))
#     # model.add(TimeDistributed(BatchNormalization(epsilon=1e-06, momentum=0.95, weights=None)))
#     model.add(TimeDistributed(Activation('tanh')))
#     model.add(TimeDistributed(Dropout(0.2093)))
#     model.add(TimeDistributed(Activation('relu')))
#     model.add(Flatten())
#     model.add(Dense(512, activation='tanh'))
#     model.add(BatchNormalization(epsilon=1e-06, momentum=0.9, weights=None))
#     model.add(Dense(n_outputs, activation='softmax'))
    adam=optimizers.Adam(lr=1e-3, beta_1=0.9, beta_2=0.999,epsilon=1e-08, amsgrad=False)
    checkpoint_filepath = '/media/naveen/nav/mat_codes/nina_DB2_codes/olsson_DB2/CNN100X12/checkpoint.hdf5'
    # model.load_weights(checkpoint_filepath) 
    checkpoint_callback = ModelCheckpoint(filepath=checkpoint_filepath,verbose=1, monitor='val_accuracy',save_weights_only=True,save_best_only=True)
    early = EarlyStopping(monitor='val_loss', min_delta=0, patience=100, verbose=0, mode='auto', baseline=None, restore_best_weights=True)
    model.compile(optimizer=adam,loss='categorical_crossentropy',metrics=['accuracy'])
    print(model.summary())
#     tf.keras.utils.plot_model(model, to_file='/media/naveen/nav/mat_codes/nina_DB1_codes/nina_prep_python/olsson/CNN20X10/Model1.png',show_shapes=True,show_layer_names=True,dpi=96)
    csv_logger = CSVLogger('/media/naveen/nav/mat_codes/nina_DB2_codes/olsson_DB2/CNN100X12/LSTM_nina_20X100.csv', append=True, separator=';')
    history = model.fit(x_train, y_train_hot, epochs=epochs, batch_size=batch_size, callbacks=[csv_logger,checkpoint_callback,lrate,early],validation_data=(x_test, y_test_hot), verbose=1)
    best_index = history.history['val_accuracy'].index(max(history.history['val_accuracy']))
    print('epoch_number',best_index+1)
    print('train accuracy and validation accuracy', history.history['accuracy'][best_index], history.history['val_accuracy'][best_index])
    model.load_weights(checkpoint_filepath) 
    _, testaccuracy = model.evaluate(x_test, y_test_hot, batch_size=batch_size, verbose=1)
    print('test_accuracy',testaccuracy)
    test_acc.append(testaccuracy)
    print(test_acc)
    print('test_mean for %d subjects:',i)
    test_mean=statistics.mean(test_acc)
    print(test_mean)

/media/naveen/nav/mat_codes/BioPatRec/olsson_prep_WOR_BP_D2/sub_wise_process_TT/S1_tr.csv
Number of columns in the dataframe: 9
Number of rows in the dataframe: 312000

/media/naveen/nav/mat_codes/BioPatRec/olsson_prep_WOR_BP_D2/sub_wise_process_TT/S1_tt.csv
Number of columns in the dataframe: 9
Number of rows in the dataframe: 156000

x_train shape:  (2079, 150, 8)
2079 training samples
y_train shape:  (2079,)
num_time_periods 150
num_sensors 8
class_list [1.0, 2.0, 3.0, 4.0, 5.0, 6.0, 7.0, 8.0, 9.0, 10.0, 11.0, 12.0, 13.0, 14.0, 15.0, 16.0, 17.0, 18.0, 19.0, 20.0, 21.0, 22.0, 23.0, 24.0, 25.0, 26.0]
x_train shape: (150, 8)
input_shape: (150, 8)
y_train shape (2079,)
New y_train shape:  (2079, 26)
x_test shape:  (1039, 150, 8)
1039 testing samples
y_test shape:  (1039,)
x_train shape:  (2079, 150, 8, 1)
x_test shape:  (1039, 150, 8, 1)
n_outputs 26
Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape         

Epoch 16/160
5/5 [==============================] - ETA: 0s - loss: 0.1784 - accuracy: 0.9986
Epoch 00016: val_accuracy did not improve from 0.84312
5/5 [==============================] - 0s 21ms/step - loss: 0.1784 - accuracy: 0.9986 - val_loss: 0.6528 - val_accuracy: 0.8412 - lr: 0.0010
Epoch 17/160
1/5 [=====>........................] - ETA: 0s - loss: 0.1746 - accuracy: 1.0000
Epoch 00017: val_accuracy did not improve from 0.84312
5/5 [==============================] - 0s 22ms/step - loss: 0.1749 - accuracy: 0.9990 - val_loss: 0.6872 - val_accuracy: 0.8229 - lr: 0.0010
Epoch 18/160
1/5 [=====>........................] - ETA: 0s - loss: 0.1715 - accuracy: 1.0000
Epoch 00018: val_accuracy did not improve from 0.84312
5/5 [==============================] - 0s 21ms/step - loss: 0.1723 - accuracy: 1.0000 - val_loss: 0.6496 - val_accuracy: 0.8335 - lr: 0.0010
Epoch 19/160
1/5 [=====>........................] - ETA: 0s - loss: 0.1716 - accuracy: 1.0000
Epoch 00019: val_accuracy improved f

1/5 [=====>........................] - ETA: 0s - loss: 0.1452 - accuracy: 1.0000
Epoch 00043: val_accuracy improved from 0.86044 to 0.86333, saving model to /media/naveen/nav/mat_codes/nina_DB2_codes/olsson_DB2/CNN100X12/checkpoint.hdf5
5/5 [==============================] - 0s 29ms/step - loss: 0.1448 - accuracy: 1.0000 - val_loss: 0.6125 - val_accuracy: 0.8633 - lr: 1.0000e-04
Epoch 44/160
5/5 [==============================] - ETA: 0s - loss: 0.1449 - accuracy: 1.0000
Epoch 00044: val_accuracy did not improve from 0.86333
5/5 [==============================] - 0s 23ms/step - loss: 0.1449 - accuracy: 1.0000 - val_loss: 0.6164 - val_accuracy: 0.8624 - lr: 1.0000e-04
Epoch 45/160
1/5 [=====>........................] - ETA: 0s - loss: 0.1444 - accuracy: 1.0000
Epoch 00045: val_accuracy did not improve from 0.86333
5/5 [==============================] - 0s 19ms/step - loss: 0.1444 - accuracy: 1.0000 - val_loss: 0.6165 - val_accuracy: 0.8585 - lr: 1.0000e-04
Epoch 46/160
1/5 [=====>......

Epoch 71/160
1/5 [=====>........................] - ETA: 0s - loss: 0.1417 - accuracy: 1.0000
Epoch 00071: val_accuracy did not improve from 0.86333
5/5 [==============================] - 0s 16ms/step - loss: 0.1416 - accuracy: 1.0000 - val_loss: 0.6249 - val_accuracy: 0.8556 - lr: 1.0000e-04
Epoch 72/160
1/5 [=====>........................] - ETA: 0s - loss: 0.1410 - accuracy: 1.0000
Epoch 00072: val_accuracy did not improve from 0.86333
5/5 [==============================] - 0s 20ms/step - loss: 0.1415 - accuracy: 1.0000 - val_loss: 0.6246 - val_accuracy: 0.8576 - lr: 1.0000e-04
Epoch 73/160
1/5 [=====>........................] - ETA: 0s - loss: 0.1409 - accuracy: 1.0000
Epoch 00073: val_accuracy did not improve from 0.86333
5/5 [==============================] - 0s 23ms/step - loss: 0.1412 - accuracy: 1.0000 - val_loss: 0.6234 - val_accuracy: 0.8585 - lr: 1.0000e-04
Epoch 74/160
1/5 [=====>........................] - ETA: 0s - loss: 0.1414 - accuracy: 1.0000
Epoch 00074: val_accurac

Epoch 99/160
5/5 [==============================] - ETA: 0s - loss: 0.1403 - accuracy: 1.0000
Epoch 00099: val_accuracy did not improve from 0.86333
5/5 [==============================] - 0s 22ms/step - loss: 0.1403 - accuracy: 1.0000 - val_loss: 0.6158 - val_accuracy: 0.8633 - lr: 1.0000e-05
Epoch 100/160
1/5 [=====>........................] - ETA: 0s - loss: 0.1401 - accuracy: 1.0000
Epoch 00100: val_accuracy did not improve from 0.86333
5/5 [==============================] - 0s 20ms/step - loss: 0.1404 - accuracy: 1.0000 - val_loss: 0.6177 - val_accuracy: 0.8614 - lr: 1.0000e-05
Epoch 101/160
1/5 [=====>........................] - ETA: 0s - loss: 0.1403 - accuracy: 1.0000
Epoch 00101: val_accuracy did not improve from 0.86333
5/5 [==============================] - 0s 16ms/step - loss: 0.1404 - accuracy: 1.0000 - val_loss: 0.6176 - val_accuracy: 0.8624 - lr: 1.0000e-05
Epoch 102/160
1/5 [=====>........................] - ETA: 0s - loss: 0.1405 - accuracy: 1.0000
Epoch 00102: val_accu

...

Epoch 26/160
1/5 [=====>........................] - ETA: 0s - loss: 0.1668 - accuracy: 1.0000
Epoch 00026: val_accuracy did not improve from 0.80077
5/5 [==============================] - 0s 18ms/step - loss: 0.1644 - accuracy: 1.0000 - val_loss: 0.7511 - val_accuracy: 0.7854 - lr: 0.0010
Epoch 27/160
1/5 [=====>........................] - ETA: 0s - loss: 0.1637 - accuracy: 1.0000
Epoch 00027: val_accuracy did not improve from 0.80077
5/5 [==============================] - 0s 19ms/step - loss: 0.1630 - accuracy: 1.0000 - val_loss: 0.8008 - val_accuracy: 0.7786 - lr: 0.0010
Epoch 28/160
1/5 [=====>........................] - ETA: 0s - loss: 0.1624 - accuracy: 1.0000
Epoch 00028: val_accuracy did not improve from 0.80077
5/5 [==============================] - 0s 18ms/step - loss: 0.1619 - accuracy: 1.0000 - val_loss: 0.7752 - val_accuracy: 0.7902 - lr: 0.0010
Epoch 29/160
1/5 [=====>........................] - ETA: 0s - loss: 0.1612 - accuracy: 1.0000
Epoch 00029: val_accuracy did not im

Epoch 54/160
1/5 [=====>........................] - ETA: 0s - loss: 0.1519 - accuracy: 1.0000
Epoch 00054: val_accuracy did not improve from 0.81232
5/5 [==============================] - 0s 18ms/step - loss: 0.1520 - accuracy: 1.0000 - val_loss: 0.7306 - val_accuracy: 0.8104 - lr: 1.0000e-04
Epoch 55/160
1/5 [=====>........................] - ETA: 0s - loss: 0.1522 - accuracy: 1.0000
Epoch 00055: val_accuracy did not improve from 0.81232
5/5 [==============================] - 0s 17ms/step - loss: 0.1522 - accuracy: 1.0000 - val_loss: 0.7322 - val_accuracy: 0.8065 - lr: 1.0000e-04
Epoch 56/160
5/5 [==============================] - ETA: 0s - loss: 0.1518 - accuracy: 1.0000
Epoch 00056: val_accuracy did not improve from 0.81232
5/5 [==============================] - 0s 21ms/step - loss: 0.1518 - accuracy: 1.0000 - val_loss: 0.7304 - val_accuracy: 0.8056 - lr: 1.0000e-04
Epoch 57/160
1/5 [=====>........................] - ETA: 0s - loss: 0.1521 - accuracy: 1.0000
Epoch 00057: val_accurac

Epoch 82/160
1/5 [=====>........................] - ETA: 0s - loss: 0.1500 - accuracy: 1.0000
Epoch 00082: val_accuracy did not improve from 0.81232
5/5 [==============================] - 0s 19ms/step - loss: 0.1503 - accuracy: 1.0000 - val_loss: 0.7267 - val_accuracy: 0.8065 - lr: 1.0000e-05
Epoch 83/160
1/5 [=====>........................] - ETA: 0s - loss: 0.1506 - accuracy: 1.0000
Epoch 00083: val_accuracy did not improve from 0.81232
5/5 [==============================] - 0s 18ms/step - loss: 0.1503 - accuracy: 1.0000 - val_loss: 0.7263 - val_accuracy: 0.8046 - lr: 1.0000e-05
Epoch 84/160
1/5 [=====>........................] - ETA: 0s - loss: 0.1501 - accuracy: 1.0000
Epoch 00084: val_accuracy did not improve from 0.81232
5/5 [==============================] - 0s 18ms/step - loss: 0.1505 - accuracy: 1.0000 - val_loss: 0.7269 - val_accuracy: 0.8075 - lr: 1.0000e-05
Epoch 85/160
1/5 [=====>........................] - ETA: 0s - loss: 0.1502 - accuracy: 1.0000
Epoch 00085: val_accurac

Epoch 110/160
1/5 [=====>........................] - ETA: 0s - loss: 0.1503 - accuracy: 1.0000
Epoch 00110: val_accuracy did not improve from 0.81232
5/5 [==============================] - 0s 18ms/step - loss: 0.1501 - accuracy: 1.0000 - val_loss: 0.7265 - val_accuracy: 0.8046 - lr: 1.0000e-05
Epoch 111/160
1/5 [=====>........................] - ETA: 0s - loss: 0.1503 - accuracy: 1.0000
Epoch 00111: val_accuracy did not improve from 0.81232
5/5 [==============================] - 0s 18ms/step - loss: 0.1499 - accuracy: 1.0000 - val_loss: 0.7274 - val_accuracy: 0.8065 - lr: 1.0000e-05
Epoch 112/160
1/5 [=====>........................] - ETA: 0s - loss: 0.1495 - accuracy: 1.0000
Epoch 00112: val_accuracy did not improve from 0.81232
5/5 [==============================] - 0s 18ms/step - loss: 0.1500 - accuracy: 1.0000 - val_loss: 0.7266 - val_accuracy: 0.8056 - lr: 1.0000e-05
Epoch 113/160
1/5 [=====>........................] - ETA: 0s - loss: 0.1498 - accuracy: 1.0000
Epoch 00113: val_acc

Epoch 138/160
1/5 [=====>........................] - ETA: 0s - loss: 0.1502 - accuracy: 1.0000
Epoch 00138: val_accuracy did not improve from 0.81232
5/5 [==============================] - 0s 18ms/step - loss: 0.1499 - accuracy: 1.0000 - val_loss: 0.7274 - val_accuracy: 0.8056 - lr: 1.0000e-06
Epoch 139/160
1/5 [=====>........................] - ETA: 0s - loss: 0.1498 - accuracy: 1.0000
Epoch 00139: val_accuracy did not improve from 0.81232
5/5 [==============================] - 0s 19ms/step - loss: 0.1498 - accuracy: 1.0000 - val_loss: 0.7272 - val_accuracy: 0.8046 - lr: 1.0000e-06
Epoch 140/160
1/5 [=====>........................] - ETA: 0s - loss: 0.1497 - accuracy: 1.0000
Epoch 00140: val_accuracy did not improve from 0.81232
5/5 [==============================] - 0s 18ms/step - loss: 0.1498 - accuracy: 1.0000 - val_loss: 0.7268 - val_accuracy: 0.8065 - lr: 1.0000e-06
Epoch 141/160
1/5 [=====>........................] - ETA: 0s - loss: 0.1501 - accuracy: 1.0000
Epoch 00141: val_acc

In [22]:
statistics.mean(test_acc)

0.8419294532607583